<font size="6"><b>INTERFACING TO DATABASES</b></font>

In [ ]:
library(data.table)
library(tidyverse)
library(DBI)
library(RSQLite)
library(dbplyr)

In [ ]:
options(repr.matrix.max.rows=20, repr.matrix.max.cols=15) # for limiting the number of top and bottom rows of tables printed 

![xkcd](../imagesbb/query2.png)

(https://xkcd.com/1409/)

The goal of this chapter is to use simple or familiar tools so that you can query relational databases to extract information.

Learning SQL is beyond the scope however getting familiar to basic SQL clauses would be useful so that you can realize the similarities between SQL and the semantics of the main tools we use in R - data.table and tidyverse.

Database management commands or `INSERT`, `DELETE` or `APPEND` queries are also beyond our scope.

But we use some utility functions / wrappers from DBI package to write into or read whole tables.

Our focus is on `SELECT` queries, or commands that translate into `SELECT` queries.

We will create and connect to an SQLite database since it is lightweight and does not require a client-server architecture which is hard to implement in this binder setting. 

# Data

We will be using World database, a sample database for PostgreSQL

The database is comprised of three tables and has demographic, economic and geographic information and statistics on countries, cities and languages

The sql dump of the database is downloaded from:

https://ftp.postgresql.org/pub/projects/pgFoundry/dbsamples/world/world-1.0/world-1.0.tar.gz

Then it was restored into PostgreSQL server.
I used the tools that we cover here to read the data into R and the three tables were combined into a list object.

RDS is a binary serialization format in base R. More to talk about serialization later:

In [ ]:
worldl <- readRDS("~/databb/rds/world.rds")

In [ ]:
worldl %>% str

In [ ]:
worldl %>% lapply(head)

The entity relationship diagram of the database can provide a summary of the fields, types and relations between tables:

![xkcd](../imagesbb/world_erd.png)

# DBI

## Create a connection

Let's first create a connection to a new SQLite database. Delete the existing one if exists:

In [ ]:
if (file.exists("~/databb/temp/world.db")) file.remove("~/databb/temp/world.db")

In [ ]:
consq <- dbConnect(RSQLite::SQLite(), "~/databb/temp/world.db")

Get the names of tables from R object:

In [ ]:
tables <- names(worldl)
tables

And write the tables into the database:

In [ ]:
for (i in tables)
{
    dbt <- worldl[[i]]
    dbWriteTable(consq, i, dbt, overwrite = T)
}

List table names in the database:

In [ ]:
tables2 <- dbListTables(consq)

In [ ]:
tables2

Now let's read the whole tables back:

In [ ]:
worldl2 <- lapply(tables, function(x) dbReadTable(consq, x))

In [ ]:
names(worldl2) <- tables

In [ ]:
worldl2 %>% str

Read the fieldnames of tables:

In [ ]:
fieldsx <- lapply(tables, function(x) dbListFields(consq, x))

In [ ]:
fieldsx

# dbplyr

In [ ]:
tables

Let's create a list of lazy connection/query objects with `tbl` to each of the tables in the database

In [ ]:
world_lazy <- lapply(tables, function(x) tbl(consq, x))

In [ ]:
names(world_lazy) <- tables

The lazy query prints only the header of the query results,

The query is executed when `collect` function is called.

A lazy query can be piped into other queries so complex queries can be formed this way:

In [ ]:
world_lazy

Now let's lazily select some of the columns from the `country` table:

In [ ]:
query01 <- world_lazy$country %>% select(-governmentform, -headofstate)

See the automatically created SQL query under the hood:

In [ ]:
query01 %>% show_query

In [ ]:
query01

Starting from the previous query, let's filter some rows:

In [ ]:
query02 <- query01 %>% filter(continent == "Europe" & lifeexpectancy > 75)

In [ ]:
query02 %>% show_query

In [ ]:
query02

And on top of the last query, let's calculate and create a new column: 

In [ ]:
query03 <- query02 %>% mutate(popdensity = population / surfacearea)

In [ ]:
query03 %>% show_query

In [ ]:
query03

Let's join the last query into selected columns of `countrylanguage` table:

In [ ]:
query04 <- query03 %>% left_join(world_lazy$countrylanguage %>% select(countrycode, language, isofficial), by = c("code" = "countrycode"))

In [ ]:
query04 %>% show_query

And summarize the number of total languages and official languages for each country into a new query `query05`:

In [ ]:
query05 <- query04 %>% group_by(code) %>% summarise(nlang = n(), noff = sum(isofficial, na.rm = T)) %>% ungroup

In [ ]:
query05 %>% show_query

In [ ]:
query05

Now let's combine the 3rd and 5th queries so that for each country now we have the number of total languages and official languages, in addition to the already selected or created columns:

In [ ]:
query06 <- query03 %>% left_join(query05, by = "code")

See how complicated the query becomes:

In [ ]:
query06 %>% show_query

In [ ]:
query06

Now let's join the city information on the capital of each country, we change some names to avoid name collision:

In [ ]:
query07 <- query06 %>% left_join(world_lazy$city %>% select(id, capitalcity = name, capital_pop = population), by = c("capital" = "id"))

See that if we were to write all these as an SQL query, it would be a huge task:

In [ ]:
query07 %>% show_query

In [ ]:
query07

Now it is time to execute all the steps and collect the results (as a tibble object by default):

In [ ]:
results <- query07 %>% collect

In [ ]:
results

Now let's combine these separete steps into three stages:

Stage 1 includes the initial steps we do with the `country` table:

In [ ]:
stage1 <- world_lazy$country %>%
select(-governmentform, -headofstate) %>%
filter(continent == "Europe" & lifeexpectancy > 75) %>%
mutate(popdensity = population / surfacearea)

Stage 2 joins Stage 1 to the countrylanguage and calculates the number of all languages and official languages:

In [ ]:
stage2 <- stage1 %>%
left_join(world_lazy$countrylanguage %>% select(countrycode, language, isofficial),
          by = c("code" = "countrycode")) %>%
group_by(code) %>%
summarise(nlang = n(), noff = sum(isofficial, na.rm = T))

In Stage 3, Stage 1 query is joined with the Stage 2 query

In [ ]:
stage3 <- stage1 %>% left_join(stage2, by = "code") %>%
left_join(world_lazy$city %>% select(id, capitalcity = name, capital_pop = population),
          by = c("capital" = "id"))

Easy dplyr verbs above create such a complex query below:

In [ ]:
stage3 %>% show_query

In [ ]:
stage3

Now collect the results again:

In [ ]:
results2 <- stage3 %>% collect

And see that combining separate steps into stages does not change the result:

In [ ]:
identical(results, results2)

# Using dbGetQuery from DBI

Now let's extract the sql_query as a character object:

In [ ]:
queryx <- stage3 %>% sql_render

In [ ]:
queryx %>% str

Pretty print the query with new lines:

In [ ]:
queryx %>% cat

Now let's execute the query using the generic function in DBI and get the results a third time:

In [ ]:
results3 <- dbGetQuery(consq, queryx)

Make the object a tibble:

In [ ]:
results3 <- as_tibble(results3)

And see that the resulsts are identical:

In [ ]:
identical(results3, results)